In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1057331,2021-03-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1057332,2021-03-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1057333,2021-03-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1057334,2021-03-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28611,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28613,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28615,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28617,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28619,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
654249,2021-03-05,Arkansas,Arkansas,5001,2002,33.00,5000,Arkansas,AR,Arkansas,State,3017804
654251,2021-03-06,Arkansas,Arkansas,5001,2005,33.00,5000,Arkansas,AR,Arkansas,State,3017804
654253,2021-03-07,Arkansas,Arkansas,5001,2005,33.00,5000,Arkansas,AR,Arkansas,State,3017804
654255,2021-03-08,Arkansas,Arkansas,5001,2005,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1057331,2021-03-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1057332,2021-03-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1057333,2021-03-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1057334,2021-03-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-08') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
412,2021-03-08,Snohomish,Washington,31439,545.00,53061,WA,County,822083,3824.31,66.30
823,2021-03-08,Cook,Illinois,479361,10011.00,17031,IL,County,5150233,9307.56,194.38
1233,2021-03-08,Orange,California,262849,4252.00,6059,CA,County,3175692,8276.90,133.89
1642,2021-03-08,Maricopa,Arizona,516720,9316.00,4013,AZ,County,4485414,11520.01,207.70
2051,2021-03-08,Los Angeles,California,1204665,22041.00,6037,CA,County,10039107,11999.72,219.55
...,...,...,...,...,...,...,...,...,...,...,...
1056866,2021-03-08,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1057014,2021-03-08,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1057131,2021-03-08,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1057244,2021-03-08,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
412,2021-03-08,Snohomish,Washington,31439,545.00,53061,WA,County,822083,3824.31,66.30,66.30,3824.31
823,2021-03-08,Cook,Illinois,479361,10011.00,17031,IL,County,5150233,9307.56,194.38,194.38,9307.56
1233,2021-03-08,Orange,California,262849,4252.00,6059,CA,County,3175692,8276.90,133.89,133.89,8276.90
1642,2021-03-08,Maricopa,Arizona,516720,9316.00,4013,AZ,County,4485414,11520.01,207.70,207.70,11520.01
2051,2021-03-08,Los Angeles,California,1204665,22041.00,6037,CA,County,10039107,11999.72,219.55,219.55,11999.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056866,2021-03-08,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1057014,2021-03-08,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1057131,2021-03-08,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1057244,2021-03-08,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
744304,2021-03-08,Clarke,Alabama,3446,53.00,1025,AL,County,23622,14588.10,224.37,224.37,14588.10,1
684009,2021-03-08,Hale,Alabama,2129,69.00,1065,AL,County,14651,14531.43,470.96,470.96,14531.43,2
535580,2021-03-08,Lowndes,Alabama,1333,51.00,1085,AL,County,9726,13705.53,524.37,524.37,13705.53,3
475036,2021-03-08,Etowah,Alabama,13427,327.00,1055,AL,County,102268,13129.23,319.75,319.75,13129.23,4
535929,2021-03-08,Marengo,Alabama,2454,57.00,1091,AL,County,18863,13009.60,302.18,302.18,13009.60,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652871,2021-03-08,Converse,Wyoming,973,16.00,56009,WY,County,13822,7039.50,115.76,115.76,7039.50,19
653218,2021-03-08,Sublette,Wyoming,681,7.00,56035,WY,County,9831,6927.07,71.20,71.20,6927.07,20
902072,2021-03-08,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,21
810996,2021-03-08,Lincoln,Wyoming,1260,11.00,56023,WY,County,19830,6354.01,55.47,55.47,6354.01,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
535580,2021-03-08,Lowndes,Alabama,1333,51.00,1085,AL,County,9726,13705.53,524.37,524.37,13705.53,3,1
684009,2021-03-08,Hale,Alabama,2129,69.00,1065,AL,County,14651,14531.43,470.96,470.96,14531.43,2,2
228241,2021-03-08,Walker,Alabama,6606,260.00,1127,AL,County,63521,10399.71,409.31,409.31,10399.71,31,3
473636,2021-03-08,Clay,Alabama,1462,54.00,1027,AL,County,13235,11046.47,408.01,408.01,11046.47,20,4
534882,2021-03-08,Crenshaw,Alabama,1473,55.00,1041,AL,County,13772,10695.61,399.36,399.36,10695.61,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576064,2021-03-08,Hot Springs,Wyoming,361,3.00,56017,WY,County,4413,8180.38,67.98,67.98,8180.38,17,19
790877,2021-03-08,Uinta,Wyoming,2076,12.00,56041,WY,County,20226,10264.02,59.33,59.33,10264.02,4,20
810996,2021-03-08,Lincoln,Wyoming,1260,11.00,56023,WY,County,19830,6354.01,55.47,55.47,6354.01,22,21
227173,2021-03-08,Teton,Wyoming,3397,9.00,56039,WY,County,23464,14477.50,38.36,38.36,14477.50,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.30,3824.31,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.30,3824.31,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.30,3824.31,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.30,3824.31,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.30,3824.31,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054839,2021-03-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1054840,2021-03-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1054841,2021-03-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1054842,2021-03-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
741471,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14588.10,39,1,2.00
741472,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14588.10,39,1,0.00
741473,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,224.37,14588.10,39,1,1.00
741474,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,224.37,14588.10,39,1,4.00
741475,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,224.37,14588.10,39,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824481,2021-03-05,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
824482,2021-03-06,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
824483,2021-03-07,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
824484,2021-03-08,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
533435,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13705.53,1,3,1.00,0.00
533436,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13705.53,1,3,0.00,0.00
533437,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13705.53,1,3,0.00,0.00
533438,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13705.53,1,3,0.00,0.00
533439,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13705.53,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531685,2021-03-05,Albany,Wyoming,3913,11.00,56001,WY,County,38880,10064.30,28.29,28.29,10074.59,23,6,1.00,0.00
531686,2021-03-06,Albany,Wyoming,3913,11.00,56001,WY,County,38880,10064.30,28.29,28.29,10074.59,23,6,0.00,0.00
531687,2021-03-07,Albany,Wyoming,3913,11.00,56001,WY,County,38880,10064.30,28.29,28.29,10074.59,23,6,0.00,0.00
531688,2021-03-08,Albany,Wyoming,3917,11.00,56001,WY,County,38880,10074.59,28.29,28.29,10074.59,23,6,4.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-08') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
272353,2021-03-08,Imperial,California,27019,658.00,6025,CA,County,181215,14909.91,363.10,363.10,14909.91,1,2,10.00,20.00
2051,2021-03-08,Los Angeles,California,1204665,22041.00,6037,CA,County,10039107,11999.72,219.55,219.55,11999.72,2,5,866.00,12.00
476388,2021-03-08,Inyo,California,1320,37.00,6027,CA,County,18039,7317.48,205.11,205.11,7317.48,3,25,8.00,0.00
60832,2021-03-08,Stanislaus,California,56885,959.00,6099,CA,County,550660,10330.33,174.15,174.15,10330.33,4,10,183.00,2.00
75389,2021-03-08,Tulare,California,48319,773.00,6107,CA,County,466195,10364.55,165.81,165.81,10364.55,5,9,116.00,3.00
28241,2021-03-08,Riverside,California,291040,3912.00,6065,CA,County,2470546,11780.39,158.35,158.35,11780.39,6,6,136.00,20.00
48424,2021-03-08,San Joaquin,California,67395,1189.00,6077,CA,County,762148,8842.77,156.01,156.01,8842.77,7,18,111.00,7.00
577399,2021-03-08,Kings,California,22211,229.00,6031,CA,County,152940,14522.69,149.73,149.73,14522.69,8,3,27.00,2.00
431828,2021-03-08,Merced,California,29557,415.00,6047,CA,County,277680,10644.27,149.45,149.45,10644.27,9,8,181.00,2.00
27505,2021-03-08,Fresno,California,96549,1488.00,6019,CA,County,999101,9663.59,148.93,148.93,9663.59,10,14,183.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1004406,2021-03-08,Lassen,California,5629,25.00,6035,CA,County,30573,18411.67,81.77,81.77,18411.67,28,1,6.00,1.00
272353,2021-03-08,Imperial,California,27019,658.00,6025,CA,County,181215,14909.91,363.10,363.10,14909.91,1,2,10.00,20.00
577399,2021-03-08,Kings,California,22211,229.00,6031,CA,County,152940,14522.69,149.73,149.73,14522.69,8,3,27.00,2.00
135819,2021-03-08,San Bernardino,California,288044,3176.00,6071,CA,County,2180085,13212.51,145.68,145.68,13212.51,11,4,177.00,23.00
2051,2021-03-08,Los Angeles,California,1204665,22041.00,6037,CA,County,10039107,11999.72,219.55,219.55,11999.72,2,5,866.00,12.00
28241,2021-03-08,Riverside,California,291040,3912.00,6065,CA,County,2470546,11780.39,158.35,158.35,11780.39,6,6,136.00,20.00
170684,2021-03-08,Kern,California,104307,942.00,6029,CA,County,900202,11587.07,104.64,104.64,11587.07,17,7,65.00,0.00
431828,2021-03-08,Merced,California,29557,415.00,6047,CA,County,277680,10644.27,149.45,149.45,10644.27,9,8,181.00,2.00
75389,2021-03-08,Tulare,California,48319,773.00,6107,CA,County,466195,10364.55,165.81,165.81,10364.55,5,9,116.00,3.00
60832,2021-03-08,Stanislaus,California,56885,959.00,6099,CA,County,550660,10330.33,174.15,174.15,10330.33,4,10,183.00,2.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
272353,2021-03-08,Imperial,California,27019,658.00,6025,CA,County,181215,14909.91,363.10,363.10,14909.91,1,2,10.00,20.00
2051,2021-03-08,Los Angeles,California,1204665,22041.00,6037,CA,County,10039107,11999.72,219.55,219.55,11999.72,2,5,866.00,12.00
476388,2021-03-08,Inyo,California,1320,37.00,6027,CA,County,18039,7317.48,205.11,205.11,7317.48,3,25,8.00,0.00
60832,2021-03-08,Stanislaus,California,56885,959.00,6099,CA,County,550660,10330.33,174.15,174.15,10330.33,4,10,183.00,2.00
75389,2021-03-08,Tulare,California,48319,773.00,6107,CA,County,466195,10364.55,165.81,165.81,10364.55,5,9,116.00,3.00
28241,2021-03-08,Riverside,California,291040,3912.00,6065,CA,County,2470546,11780.39,158.35,158.35,11780.39,6,6,136.00,20.00
48424,2021-03-08,San Joaquin,California,67395,1189.00,6077,CA,County,762148,8842.77,156.01,156.01,8842.77,7,18,111.00,7.00
577399,2021-03-08,Kings,California,22211,229.00,6031,CA,County,152940,14522.69,149.73,149.73,14522.69,8,3,27.00,2.00
431828,2021-03-08,Merced,California,29557,415.00,6047,CA,County,277680,10644.27,149.45,149.45,10644.27,9,8,181.00,2.00
27505,2021-03-08,Fresno,California,96549,1488.00,6019,CA,County,999101,9663.59,148.93,148.93,9663.59,10,14,183.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,363.10,14909.91,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,363.10,14909.91,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,363.10,14909.91,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,363.10,14909.91,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,363.10,14909.91,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3562,03/05/21,Lassen,5623,19.00,18392.05,62.15,81.77,18411.67,28,1,0.00,0.00
3563,03/06/21,Lassen,5623,24.00,18392.05,78.50,81.77,18411.67,28,1,0.00,5.00
3564,03/07/21,Lassen,5623,24.00,18392.05,78.50,81.77,18411.67,28,1,0.00,0.00
3565,03/08/21,Lassen,5629,25.00,18411.67,81.77,81.77,18411.67,28,1,6.00,1.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,363.10,14909.91,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,363.10,14909.91,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,363.10,14909.91,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,363.10,14909.91,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,363.10,14909.91,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3636,03/05/21,Fresno,96023,1466.00,9610.94,146.73,148.93,9663.59,10,14,130.00,8.00
3637,03/06/21,Fresno,96184,1488.00,9627.05,148.93,148.93,9663.59,10,14,161.00,22.00
3638,03/07/21,Fresno,96366,1488.00,9645.27,148.93,148.93,9663.59,10,14,182.00,0.00
3639,03/08/21,Fresno,96549,1488.00,9663.59,148.93,148.93,9663.59,10,14,183.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)